#### Q3 Train a Pure CNN with less than 10000 trainable parameters using the MNISTDataset having minimum validation accuracy of 99.40%
Note -
1. Code comments should be given for proper code understanding.
2. Implement in both PyTorch and Tensorflow respectively

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [5]:

(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [9]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape and normalize input data
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# One-hot encode the target labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the CNN architecture
model = Sequential([
    Conv2D(8, (3, 3), activation='tanh', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(16, (3, 3), activation='tanh'),
    MaxPooling2D((2, 2)),
#    Dropout(0.5),
    Flatten(),
    Dense(20, activation='relu'),
    Dense(10, activation='softmax')
])
# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 8)         80        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 8)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 16)        1168      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 20)               

In [10]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=25, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model on the test set
_, accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", accuracy*100)


Epoch 1/25
1875/1875 [==============================] - 12s 4ms/step - loss: 0.2926 - accuracy: 0.9139 - val_loss: 0.1056 - val_accuracy: 0.9668
Epoch 2/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0953 - accuracy: 0.9711 - val_loss: 0.0651 - val_accuracy: 0.9790
Epoch 3/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0714 - accuracy: 0.9786 - val_loss: 0.0550 - val_accuracy: 0.9826
Epoch 4/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0613 - accuracy: 0.9813 - val_loss: 0.0519 - val_accuracy: 0.9829
Epoch 5/25
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0528 - accuracy: 0.9838 - val_loss: 0.0474 - val_accuracy: 0.9842
Epoch 6/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0466 - accuracy: 0.9856 - val_loss: 0.0487 - val_accuracy: 0.9845
Epoch 7/25
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0414 - accuracy: 0.9871 - val_loss: 0.0410 - val_accuracy

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SubsetRandomSampler

# Set device check for availbility of GPU 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST dataset
train_dataset = MNIST(root="./data", train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root="./data", train=False, download=True, transform=ToTensor())

# Define indices for train-validation split
num_samples = len(train_dataset)
indices = list(range(num_samples))
split = int(0.8 * num_samples)  # 80% train, 20% validation
train_indices, val_indices = indices[:split], indices[split:]

# Create data loaders
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_sampler)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(16 * 5 * 5, 20)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(20, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.dropout(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Create the model instance
model = Net().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 25
best_val_acc = 0.0

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    train_correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        train_correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_acc = 100.0 * train_correct / len(train_indices)

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_acc = 100.0 * val_correct / len(val_indices)

    print(f"Epoch {epoch + 1}/{num_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

    # Save the model with the best validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")

# Load the best model weights
model.load_state_dict(torch.load("best_model.pt"))

# Evaluation on test set
model.eval()
test_correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()

test_acc = 100.0 * test_correct / len(test_dataset)

print(f"Test Accuracy: {test_acc:.2f}%")

100%|██████████| 9912422/9912422 [00:00<00:00, 191307500.53it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 100946411.52it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 71129924.16it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7290673.08it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1/25: Train Loss: 782.2147, Train Acc: 83.28%, Val Loss: 47.2231, Val Acc: 96.32%
Epoch 2/25: Train Loss: 305.3259, Train Acc: 93.79%, Val Loss: 33.6214, Val Acc: 97.51%
Epoch 3/25: Train Loss: 252.0562, Train Acc: 94.85%, Val Loss: 28.8628, Val Acc: 97.85%
Epoch 4/25: Train Loss: 228.1653, Train Acc: 95.33%, Val Loss: 26.7154, Val Acc: 97.96%
Epoch 5/25: Train Loss: 206.9610, Train Acc: 95.70%, Val Loss: 25.8790, Val Acc: 97.97%
Epoch 6/25: Train Loss: 193.2620, Train Acc: 95.92%, Val Loss: 22.3346, Val Acc: 98.32%
Epoch 7/25: Train Loss: 178.4615, Train Acc: 96.33%, Val Loss: 22.6066, Val Acc: 98.30%
Epoch 8/25: Train Loss: 172.8802, Train Acc: 96.35%, Val Loss: 23.0035, Val Acc: 98.35%
Epoch 9/25: Train Loss: 165.2192, Train Acc: 96.43%, Val Loss: 20.7274, Val Acc: 98.40%
Epoch 10/25: Train Loss: 155.4549, Train Acc: 96.66%, Val Loss: 19.3646, Val Acc: 98.50%
Epoch 11/25: Train Loss: 153.7626, Train Acc